# From the 🤗 HuggingFace Hub to Vertex AI on GPU

In this notebook, we will go through the same steps as mentioned before in the following notebooks: `01-build-custom-inference-images.ipynb` and `02-upload-register-and-deploy-models.ipynb`, similarly to `03-from-hub-to-vertex-ai.ipynb` which is an end to end guide on how to deploy models from the HuggingFace Hub into Vertex AI on CPU, this notebook will only add replacements/modifications were needed in order to use the GPU.

For a more detailed explanation on each step, please consider having a look at the notebooks mentioned before.

## Installation

* `gcloud` CLI needs to be installed and logged in the project that will be used. See the installation notes at https://cloud.google.com/sdk/docs/install

* `docker` needs to be installed locally, and up and running, since it will be used to build the CPR images before pushing those to the container registry. See the installation notes at https://docs.docker.com/engine/install/

* `google-cloud-aiplatform` Python library is required to programatically build the CPR image, to define the custom prediction code via a custom `Predictor`, to register and deploy the model to an endpoint in Vertex AI, and to run the online prediction on it.

    `pip install google-cloud-aiplatform --upgrade`

* `git lfs` needs to be installed for pulling / cloning models from the HuggingFace Hub. See the installation notes at https://git-lfs.com/.

## Setup

To successfully run the code below, you will need to be authenticated into your Google Cloud account and the following variable values must be set in advance:

* `REGION` is the region where the resources will be hosted in.
* `PROJECT_ID` is the identifier of the project in Google Cloud.
* `REPOSITORY` is the directory where the Docker images will be uploaded to.
* `IMAGE` is the name of the Docker image (without tag).
* `TAG` is that tag of the Docker image.
* `BUCKET_NAME` is the name of the bucket were the model will be / has been uploaded to.
* `BUCKET_URI` is the full path to the `model.tar.gz` file in Cloud Storage.

In [ ]:
REGION = "europe-west4"
PROJECT_ID = "huggingface-cloud"
REPOSITORY = "custom-inference-gpu"
IMAGE = "huggingface-pipeline-gpu"
TAG = "py310-cu12.3-torch-2.2.0-transformers-4.38.1"
BUCKET_NAME = "huggingface-cloud"
BUCKET_URI = f"gs://{BUCKET_NAME}/bart-large-mnli/model.tar.gz"

_**Note**: before choosing a region, you should ensure that the region has access to machines with GPU accelerators, otherwise, the deployment will fail as some zones won't have access to those i.e. `europe-west4` has but `europe-west9` hasn't. More information at https://cloud.google.com/vertex-ai/pricing#prediction-prices._

---

## Custom Prediction Routine (CPR) using 🤗 `tranformers.pipeline`

There are some things to take into consideration before developing the `HuggingFacePredictor` that will be in charge of running the inference for the model above, as we need to ensure that it uses the GPU accelerator. Since we're using the `pipeline` method from `transformers`, we can simply rely on the `device_map` argument that has an `auto` option (powered by `accelerate`) that will automatically define the device mapping for the given model.

For more information check [`01-build-custom-inference-images.ipynb`](./01-build-custom-inference-images.ipynb).

In [ ]:
!mkdir huggingface_predictor_gpu

In [ ]:
%%writefile huggingface_predictor_gpu/predictor.py
import os
import logging
import tarfile
from typing import Any, Dict

from transformers import pipeline

from google.cloud.aiplatform.prediction.predictor import Predictor
from google.cloud.aiplatform.utils import prediction_utils

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)


class HuggingFacePredictor(Predictor):
    def __init__(self) -> None:
        pass
    
    def load(self, artifacts_uri: str) -> None:
        """Loads the preprocessor and model artifacts."""
        logger.debug(f"Downloading artifacts from {artifacts_uri}")
        prediction_utils.download_model_artifacts(artifacts_uri)
        logger.debug("Artifacts successfully downloaded!")
        os.makedirs("./model", exist_ok=True)
        with tarfile.open("model.tar.gz", "r:gz") as tar:
            tar.extractall(path="./model")
        logger.debug(f"HF_TASK value is {os.getenv('HF_TASK')}")
        self._pipeline = pipeline(os.getenv("HF_TASK", None), model="./model", device_map="auto")
        logger.debug("`pipeline` successfully loaded!")
        logger.debug(f"`pipeline` is using device={self._pipeline.device}")

    def predict(self, instances: Dict[str, Any]) -> Dict[str, Any]:
        return self._pipeline(**instances)

Also note that since we're using `device_map` from 🤗 `accelerate`, we will need to also add `accelerate` as a dependency within the `requirements.txt` file.

In [ ]:
%%writefile huggingface_predictor_gpu/requirements.txt
torch==2.2.0
transformers==4.38.1
accelerate==0.27.0

Then we can already build the new Docker image with support for GPU inference, and we will use a custom image from the Docker Hub `alvarobartt/torch-gpu:py310-cu12.3-torch-2.2.0` which has been put together so that it's easier to use within Vertex AI, so that the user just needs to add the dependencies within `requirements.txt` without caring about the CUDA drivers and such.

Anyway, everyone can create their own custom Docker image, but it should have the following:

* `python` installed under the alias of `python`, not `python3`
* `pip` installed under the alias of `pip`, not `pip3`
* Needs to be built with `--platform=linux/amd64` to work on Vertex AI (if not using Linux already)

_**Note**: I will soon add another tutorial on how to create custom Docker images and directly push those to Google's Container Registry without having to rely on `LocalModel.build_cpr_model`, in an easier and more flexible way._

In [ ]:
import os
from google.cloud.aiplatform.prediction import LocalModel

from huggingface_predictor_gpu.predictor import HuggingFacePredictor

local_model = LocalModel.build_cpr_model(
    "huggingface_predictor_gpu",
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}:{TAG}",
    predictor=HuggingFacePredictor,
    requirements_path="huggingface_predictor_gpu/requirements.txt",
    base_image="--platform=linux/amd64 alvarobartt/torch-gpu:py310-cu12.3-torch-2.2.0 AS build",
)

In [ ]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

In [ ]:
!gcloud artifacts repositories create custom-inference-gpu --repository-format=docker --location={REGION}

In [ ]:
local_model.push_image()

![](https://raw.githubusercontent.com/alvarobartt/vertex-ai-huggingface/main/online-prediction/assets/docker-artifact-registry.png)

---

## Upload model from the 🤗 Hub

First we need to decide which model from the HuggingFace Hub we want to use, in this case, we will be using `facebook/bart-large-mnli` which is a zero-shot classification model.

In order to do so, we will pull the model from the HuggingFace Hub using `git pull`, which requires `git lfs` to be installed in advanced, in order to also pull the large files from the repository.

For more information check [`02-upload-register-and-deploy-models.ipynb`](./02-upload-register-and-deploy-models.ipynb).

In [ ]:
!git lfs install
!git clone https://huggingface.co/facebook/bart-large-mnli

In [ ]:
!ls

In [ ]:
!cd bart-large-mnli/ && tar zcvf model.tar.gz --exclude flax_model.msgpack --exclude pytorch_model.bin --exclude rust_model.ot * && mv model.tar.gz ../

In [ ]:
!gcloud config set storage/parallel_composite_upload_enabled True
!gcloud storage cp model.tar.gz $BUCKET_URI

In [ ]:
!gcloud storage ls --recursive gs://{BUCKET_NAME}/bart-large-mnli

![](https://raw.githubusercontent.com/alvarobartt/vertex-ai-huggingface/main/online-prediction/assets/model-cloud-storage.png)

## Register model in Vertex AI

Once the model is uploaded to GCS and that the CPR image has been pushed to Google's Container Registry, then we can already register the model in Vertex AI.

For more information check [`02-upload-register-and-deploy-models.ipynb`](./02-upload-register-and-deploy-models.ipynb).

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

In [ ]:
!gcloud auth login
!gcloud auth application-default login

In [ ]:
model = aiplatform.Model.upload(
    display_name="bart-large-mnli",
    artifact_uri="gs://huggingface-cloud/bart-large-mnli",
    serving_container_image_uri=local_model.get_serving_container_spec().image_uri,
    serving_container_environment_variables={
        "HF_TASK": "zero-shot-classification",
        # Optional env var so that `uvicorn` only runs the model in 1 worker
        # "VERTEX_CPR_WEB_CONCURRENCY": 1,
    },
)

![](https://raw.githubusercontent.com/alvarobartt/vertex-ai-huggingface/main/online-prediction/assets/model-registry-vertex-ai.png)

_**Note**: the `deploy` method will take a while ~15-20 minutes in order to deploy the model in Vertex AI as an endpoint._

In this case, since we want to run the inference on GPU, we need to ensure that the `machine_type` matches a machine with GPU, those being the ones from the G2 Series. Also, as already mentioned at the beginning of the notebook, some regions/zones may not have access to GPU accelerators, so double check that the region/zone that you're using has access to the machine.

In [ ]:
endpoint = model.deploy(
    machine_type="g2-standard-4",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
)

![](https://raw.githubusercontent.com/alvarobartt/vertex-ai-huggingface/main/online-prediction/assets/endpoint-deployment-vertex-ai.png)

---

## Run online predictions on Vertex AI

Finally, we can proceed to run the online predictions on Vertex AI using their Python client, which will basically send the requests to the running endpoint, and we will also be able to closely monitor it via Google Cloud Logging service.

In [ ]:
import json
from google.api import httpbody_pb2
from google.cloud import aiplatform_v1

prediction_client = aiplatform_v1.PredictionServiceClient(
    client_options={"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
)

data = {
    "sequences": "Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app.",
    "candidate_labels": ["mobile", "website", "billing", "account access"],
}

json_data = json.dumps(data)

http_body = httpbody_pb2.HttpBody(
    data=json_data.encode("utf-8"),
    content_type="application/json",
)

request = aiplatform_v1.RawPredictRequest(
    endpoint=endpoint.resource_name,
    http_body=http_body,
)

response = prediction_client.raw_predict(request)
json.loads(response.data)

---

## Resource clean-up

Finally, we can release the resources we've allocated / created using the following `delete` methods over both the `endpoint` and the `model` variables in Vertex AI, and also the following `gcloud` CLI commands to remove both the Docker image from the Container Registry and the HuggingFace model from the Cloud Storage.

In [ ]:
endpoint.delete(force=True)
model.delete()

In [ ]:
!gcloud artifacts docker images delete --quiet --delete-tags {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}
!gcloud storage rm -r $BUCKET_URI